# Geospatial Data Structures & Algorithms

*In spatial data science, and more generally in the computational sciences, we fundamentally must deal with abstractions and representaions of real-world phenomena. The predominant abstractions we use are features and layers, but even these are abstractions of several core fundamental data structures and algorithms that we use to perform computational geometry. These examples will show basic data structures and algorithms and well as geometric data structures and algorithms that are commonly used on spatial data.*

Last updated: June 5, 2024

# Geometric Data Structures

Expanding upon the more typical data structures used, there are also specialised data structures used to represent, store, and query spatial data. In this section, we will look at basic implementations of geometric objects, as well as the building blocks of spatial indexes.

## Basic Geometric Data Structures

The most fundamental geometric data structures are often called primitives, which are points, polygons, and lines for vector data, in addition to an array/raster. In this section, we will look at a bare bones implementation of points, lines, and polygons to see how they build off one another.

In [11]:
# Implement a point class
class Point:
    def __init__(self, x: float, y: float):
        # Each point has an x and y value
        self.x = x
        self.y = y
    
    # Define a helper method to see if points are identical
    def __eq__(self, other):
        # Check if x and y are the same
        return self.x == other.x and self.y == other.y

# Implement a line class
class Line:
    def __init__(self, points: list[Point]):
        # Each line is made of a list of points
        self.points = points
        
        # Check if first and last points are identical
        # If they are, it should be a polygon
        if points[0] == points[-1]:
            raise Exception("Line is closed. Should be polygon.")

# Implement a polygon class
class Polygon:
    def __init__(self, points: list[Point]):
        # Each line is made of a list of points
        self.points = points
        
        # Check if first and last points are identical
        # If they aren't, add copy of first point to end
        if points[0] != points[-1]:
            points.append(points[0])
            print("Closing polygon.")
        
        # Make sure there are at least three points
        if len(points) < 3:
            raise Exception("Invalid polygon. Need at least three points.")

In [10]:
# Create a point
point_1 = Point(1.0, 2.0)

print(f"X is {point_1.x} and Y is {point_1.y}")

X is 1.0 and Y is 2.0


In [12]:
# Create a line
point_2 = Point(1.0, 2.0)
point_3 = Point(3.0, 4.0)

line_1 = Line([point_2, point_3])

for vertex in line_1.points:
    print(f"Vertex: X={vertex.x} Y={vertex.y}")

Vertex: X=1.0 Y=2.0
Vertex: X=3.0 Y=4.0


In [13]:
# Create a closed (invalid) line
point_4 = Point(1.0, 2.0)

line_2 = Line([point_2, point_3, point_4])

Exception: Line is closed. Should be polygon.

In [16]:
# Create a polygon
polygon_1 = Polygon([point_2, point_3, point_4])

for vertex in polygon_1.points:
    print(f"Vertex: X={vertex.x} Y={vertex.y}")

Vertex: X=1.0 Y=2.0
Vertex: X=3.0 Y=4.0
Vertex: X=1.0 Y=2.0


In [17]:
# Create a polygon that needs to be closed
polygon_2 = Polygon([point_2, point_3])

for vertex in polygon_2.points:
    print(f"Vertex: X={vertex.x} Y={vertex.y}")

Closing polygon.
Vertex: X=1.0 Y=2.0
Vertex: X=3.0 Y=4.0
Vertex: X=1.0 Y=2.0


In [18]:
# Create an invalid polygon
polygon_3 = Polygon([point_2])

for vertex in polygon_3.points:
    print(f"Vertex: X={vertex.x} Y={vertex.y}")

Exception: Invalid polygon. Need at least three points.

## Advanced Geospatial Data Structures: R-Trees

There are also more advanced data structures that can be used in spatial data science, not only including ways to represent different types of data (e.g., TIN, Space Time Cube, etc...) but also ways to index spatial data. One common way to create a spatial index is to use an R-Tree, which is a type of tree data structure specifically intended for spatial/multidimensional data. Below we will see a basic implementation of how an R-Tree can be created to efficiently query spatial data.

This implementation is not optimized and provides a very basic example, but works by doing the following.

- Each node of the tree indicates an area represented as a Minimum Bounding Rectangle (MBR) used to store the points within the given MBR.
- Each node may also consist of up to four other nodes (meaning they are nested as children of a given node)
- In order to insert data into the tree, the tree must find the correct node to insert it into, and if need be, will also need to split a node into multiple nodes to ensuer that there are no more than four children for the given node 
- In order to query the points within an MBR on the tree, each node's MBR is checked to see if it intersects with the query MBR. If it does, then the children nodes' MBRs are also checked (unless it is a leaf, in which case the points are then added to the result)

In [1]:
# Class for a Minimum Bounding Rectangle (MBR)
class Rectangle:
    # Each MBR will have a min/max x & y
    def __init__(self, x_min: float, y_min: float, x_max: float, y_max: float):
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max

    # Method to check if two MBRs intersect at all (do corners overlap at all?)
    def intersects(self, other):
        return not (self.x_max < other.x_min or self.x_min > other.x_max or self.y_max < other.y_min or self.y_min > other.y_max)

    # Method to check if point is contained within the MBR
    def contains(self, point: tuple):
        x, y = point
        return self.x_min <= x <= self.x_max and self.y_min <= y <= self.y_max
    
    # Method to expand the corners of the MBR
    def expand(self, point: tuple):
        x, y = point
        new_x_min = min(self.x_min, x)
        new_y_min = min(self.y_min, y)
        new_x_max = max(self.x_max, x)
        new_y_max = max(self.y_max, y)
        return Rectangle(new_x_min, new_y_min, new_x_max, new_y_max)

    # Method to calculate the area of the MBR
    def area(self):
        return (self.x_max - self.x_min) * (self.y_max - self.y_min)

In [5]:
# Class for R-Tree Node
class Node:
    # Creates new node with given MBR
    def __init__(self, mbr: Rectangle, children: list | None = None, is_leaf: bool = True):
        self.mbr = mbr
        self.children = children or []
        self.is_leaf = is_leaf

    # Method to insert a new point
    def insert(self, point: tuple):
        if self.is_leaf:
            self.children.append(point)
            self.mbr = self.mbr.expand(point)
            # Only allowing up to four children
            if len(self.children) > 4:
                self.split()
        else:
            best_child = min(self.children, key=lambda child: child.get_expansion_area(point))
            best_child.insert(point)
            self.mbr = self.mbr.expand(point)

    # Method to calculate the difference in area from the original MBR to the expanded MBR
    def get_expansion_area(self, point: tuple):
        new_mbr = self.mbr.expand(point)
        return new_mbr.area() - self.mbr.area()

    # Method to split node, very basic way but not optimal
    def split(self):
        midpoint = len(self.children) // 2
        left_children = self.children[:midpoint]
        right_children = self.children[midpoint:]
        
        left_mbr = Rectangle(
            min(p[0] for p in left_children),
            min(p[1] for p in left_children),
            max(p[0] for p in left_children),
            max(p[1] for p in left_children)
        )

        right_mbr = Rectangle(
            min(p[0] for p in right_children),
            min(p[1] for p in right_children),
            max(p[0] for p in right_children),
            max(p[1] for p in right_children)
        )

        self.children = [
            Node(left_mbr, left_children, self.is_leaf),
            Node(right_mbr, right_children, self.is_leaf)
        ]
        
        self.is_leaf = False

    # Method to search for points that intersect the MBR for the node
    def query(self, rect: Rectangle, results: list | None = None):
        if results is None:
            results = []
        
        if self.mbr.intersects(rect):
            if self.is_leaf:
                for point in self.children:
                    if rect.contains(point):
                        results.append(point)
            else:
                for child in self.children:
                    child.query(rect, results)
        
        return results

In [3]:
# Class for implementing the R-Tree
class RTree:
    # Creates R-Tree with root node as MBR of everything (to infinity in all directions)
    def __init__(self):
        self.root = Node(Rectangle(float('inf'), float('inf'), float('-inf'), float('-inf')))
    
    # Method to insert point into the root node
    def insert(self, point: tuple):
        self.root.insert(point)
    
    # Method to search specific MBR for points
    def query(self, rect: Rectangle):
        return self.root.query(rect)

In [10]:
# Creating empty tree
r_tree = RTree()

# Creating points and inserting into tree
points = [(2, 3), (5, 4), (9, 6), (4, 7), (8, 1), (7, 2)]
for point in points:
    r_tree.insert(point)

# Querying points within the MBR provided
print(r_tree.query(Rectangle(3, 2, 10, 5)))

# Another query with a rectangle that shouldn't produce any points
print(r_tree.query(Rectangle(-1, -1, 0, 0)))

[(5, 4), (7, 2)]
[]


# Geometric Algorithms

In addition to specialized data structures for geospatial data, there are also specialised algorithms to use on these data structures. In this section, a few examples will be explored, specifically the area of a polygon and convex hull generation will be implemented.

## Advanced Geospatial/Geometric Algorithm: Area of a Polygon

Calculating the area of polygons is a common task in spatial data science, but when the polygons are not simple (e.g., rectanges, triangles, circles, etc...) the method to calculate the area becomes more difficult than a simple equation. In this example, an algorithm known as the Shoelace Algorithm will be used to calculate the area of an irregular polygon.

In this example, we will use a nested list of tuples (points, or X/Y coordinates) to represent the vertices of the Polygon. They need to be listed in counter-clockwise order.

In [12]:
def polygon_area(polygon: list):
    # Calculate number of vertices
    n_vertices = len(polygon)
    
    # Storing sums
    sum_1 = 0
    sum_2 = 0
    
    # Loop through vertices
    for i in range(0, n_vertices - 1):
        # Add the product of the X of the ith vertex times the Y of the next vertex, to sum_1
        sum_1 += polygon[i][0] *  polygon[i+1][1]
        
        # Add the product of the Y of the ith vertex times the X of the next vertex, to sum_2
        sum_2 += polygon[i][1] *  polygon[i+1][0]
    
    # Add the remaining values not included in the loop
    # Last X with first Y
    sum_1 += polygon[n_vertices - 1][0] * polygon[0][1]
    
    # First X with last Y
    sum_2 += polygon[0][0] * polygon[n_vertices - 1][1]
    
    # Calculate the absolute value of the difference betweeb sums 1 and 2, divided by 2
    area = abs(sum_1 - sum_2) / 2
    
    return area

In [15]:
# Calling function with easy example
polygon_area(
    [
        (0, 0),
        (1, 0),
        (1, 1),
        (0, 1)
    ]
)

1.0

In [14]:
# Calling function with more complex polygon
polygon_area(
    [
        (2, 7),
        (10, 1),
        (8, 6),
        (11, 7),
        (7, 10)
    ]
)

32.0
